%load_ext autoreload
%autoreload 2

In [44]:
import pandas as pd
import numpy as np
import sys
sys.path.append('./')
import xiadan_check
import myModule
import imp
imp.reload(xiadan_check)
imp.reload(myModule)
import time

In [45]:
pathway = './'
all_files  = xiadan_check.get_files(pathway, "个性化","20181011")   ##给路径
df1  = xiadan_check.read_xlsx(all_files , 0 )## read all files sheet1

In [46]:
import datetime
def changetype1(check_str):
    value = 1 if type(check_str) == datetime.datetime else 0
    return value
def changetype2(check_str):
    value = 0 if type(check_str) == datetime.datetime else 1
    return value

In [47]:
date = pd.to_datetime(time.strftime('%Y-%m-%d',time.localtime(time.time())))
stat_week = '10月2周'
## df1:原始项目；
## df2:有效反馈项目，即排除驳回、暂停、沟通中
## df3:在线项目，基于df2
## df4:完成项目，基于df2
## df5:延期项目，基于df3中未交付延期、df4

myModule.add_week(df1,'下单时间')                                              #以下单时间定义week_number
myModule.upgrade_df(df1, [['反馈周期', ['评估反馈时间','下单时间'], '-']])     #增加“反馈周期”列
tt = lambda x : getattr(x , 'days')                                            #更换“days”类型
[myModule.retype(df1, i , tt) for i in ['反馈周期']] 

df2 = df1[df1['预计交付时间'].apply(lambda x: type(x) == datetime.datetime)]   #根据“预计交付时间”筛选有效反馈项目列表
df2['预计交付时间'] = df2['预计交付时间'].apply(pd.to_datetime)                #转换时间类型
myModule.upgrade_df(df2, [['预计周期', ['预计交付时间','评估反馈时间'], '-']]) #增加“预计周期”列
[myModule.retype(df2, i , tt) for i in ['预计周期']]                           #更换“days”类型

df3 = df2[df2['实际交付时间'].apply(lambda x: x == 0)]                          #根据“实际交付时间”筛选在线项目列表

df4 = df2[df2['实际交付时间'].apply(lambda x: type(x) == datetime.datetime)]    #根据“实际交付时间”筛选已交付项目列表
df4['实际交付时间'] = df4['实际交付时间'].apply(pd.to_datetime)                 #转换时间类型
myModule.upgrade_df(df4, [['实际周期', ['实际交付时间','评估反馈时间'], '-'],   #增加“实际周期”、“延期天数”列
                         ['延期天数',['实际交付时间','预计交付时间'], '-']])
[myModule.retype(df4, i , tt) for i in ['实际周期','延期天数']]                 #更换“days”类型
myModule.add_week(df4,'实际交付时间')                                           #以实际交付时间定义week_number

tmp = df3[df3['预计交付时间'].apply(lambda x: date>x)]                          #未交付延期项目
tmp['实际周期'] = 1000
tmp['延期天数'] = 1000
df5 = df4.append(tmp)
myModule.add_week(df5,'预计交付时间')                                           #以预计交付时间定义week_number

## 分析组人员列表
fenxi = ["施毓芷","闻少楠","杨宇","刘永强","刘梦苑","何胜","陈碧雯","王霞","卢楠","冯敬岩","朱慧萍","赵倩","解飞","宋少宇"]

/home/jiankong/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/jiankong/jupyter/chart/bin/supplment_analysis/myModule.py:87: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/jiankong/jupyter/chart/bin/supplment_analysis/myModule.py:129: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexi

# 评估情况统计

In [48]:
df1.tail()

,序号,流水号,个性化分析名称,项目管理,接入时间,下单时间,信息负责人,评估反馈时间,预计交付时间,数据迁回花费时间（天）,实际交付时间,评估人时（h）,评估机时(h),是否在库里,库里周期（天）,项目管理交付时间,备注,week_number,反馈周期
256,1218,20181011-015,PM-HL170060-02东北农大12个大豆lncRNA建库测序分析任务单-02,赵景华,2018-10-11,2018-10-11,赵倩,2018-10-12,0,0,0,0,0.0,0,0.0,0,0,10月2周,1
257,1219,20181011-016,PM-GD160093-04中二院3个人样品转录组建库测序分析任务单-02,古艳芳,2018-10-11,2018-10-11,赵倩,2018-10-12,2018-10-19 00:00:00,0,0,0,0.0,0,0.0,0,0,10月2周,1
258,1220,20181011-017,PM-GD160093-01中山二院3个人有参转录组建库测序自动化分析任务单-01,古艳芳,2018-10-11,2018-10-11,王霞,2018-10-12,2018-10-19 00:00:00,0,0,0,0.0,0,0.0,0,0,10月2周,1
259,1221,20181011-018,PM-GD160093-06中山二院6个人有参转录组建库测序分析任务单-快速产品-01,古艳芳,2018-10-11,2018-10-11,王霞,2018-10-12,2018-10-19 00:00:00,0,0,0,0.0,0,0.0,0,0,10月2周,1
260,1222,20181011-019,FJ170076-07福建农林大学215个金鱼数据只分析任务单-VIP-02,古艳芳,2018-10-11,2018-10-11,刘梦苑,2018-10-12,2018-10-18 00:00:00,0,0,0,0.0,0,0.0,0,0,10月2周,1


## 接入项目数统计

### 整体接入项目数趋势图

In [49]:
ss = myModule.format(df1,'week_number',['流水号','个性化分析名称','项目管理','预计交付时间'])
ss.df['接入项目数'] = list(map(changetype2,ss.df['流水号']))
ss.df['有效接入项目数'] = list(map(changetype1,ss.df['预计交付时间']))
ss.df['驳回暂停沟通项目数'] = list(map(changetype2,ss.df['预计交付时间']))
tt = myModule.format(ss.df,'week_number',['有效接入项目数','驳回暂停沟通项目数','接入项目数'])
pp = myModule.sort_by_index(tt.groupby('sum'))
myModule.myPlot(pp).plot('line')

/home/jiankong/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/jiankong/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/jiankong/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

### 个人接入项目数分布图

In [50]:
ss = myModule.format(df1 ,'week_number' ,['流水号'])
ss.add_group('信息负责人')
pp = myModule.sort_by_index(ss.groupby('count'))
myModule.myPlot(pp).plot('stack_bar')

## 评估周期情况统计

### 预计周期趋势图

In [51]:
# 平均评估周期，建议添加最长评估周期、25%、50%、80%项目评估周期的指标
ss = myModule.format(df2 ,'week_number' ,['预计周期'])
pp = myModule.sort_by_index(ss.groupby('mean'))
myModule.myPlot(pp).plot('line')

### 预计周期>7日项目列表

In [52]:
ss = myModule.format(df2,'week_number',['流水号','个性化分析名称','信息负责人','预计周期'])
ss.select_by_value('week_number',stat_week)
ss.judge_and_replace('预计周期',7)
ss.select_by_value('预计周期_bool',True)
tt = myModule.format(ss.df,'week_number',['流水号','个性化分析名称','信息负责人','预计周期']).df
tt.sort_values(by = '预计周期',axis = 0,ascending = False)   #预计周期从大到小排序

,week_number,流水号,个性化分析名称,信息负责人,预计周期
198,10月2周,20181008-001,PM-YWHB180262-01华科协和医院2个人全基因组重测序建库测序分析任务单-快速产品-01,车天栋,20
244,10月2周,20181011-003,YN180155-01云南大学1个果蝇基因组三代DNA建库测序过滤任务单-02,占伟,14
253,10月2周,20181011-012,HA160001-22河南农大4个玉米denovo10K文库测序分析任务单-02,占伟,14
254,10月2周,20181011-013,HA160001-22河南农大4个玉米denovo10K文库测序分析任务单-01,占伟,11
203,10月2周,20181008-006,PM-BJ180807-01医科院药物所15个人有参转录组建库测序分析任务单-01,冯敬岩,10
221,10月2周,20181009-013,SD170174-04山东农大1个芒属植物Hi-C辅助基因组组装只分析任务单,刘江,9
207,10月2周,20181008-010,PM-GD170317-01深圳市人民医院16个人6mA建库测序分析任务单-02,宋少宇,8
222,10月2周,20181009-014,PM-YWBJ181246-01阜外医院40个大鼠有参普通转录组建库测序分析任务单-快速产品-01,刘梦苑,8
223,10月2周,20181009-015,BJ181246-02阜外医院8个大鼠有参普通转录组建库测序分析任务单-快速产品-01,刘梦苑,8
234,10月2周,20181010-007,PM-BJ181079-03动物所9个小鼠Bisulfite甲基化建库测序分析任务单-04,韩少怀,8


## 反馈延期情况统计

### 反馈延期率趋势图

In [53]:
ss = myModule.format(df1 ,'week_number' ,['流水号','反馈周期'])
ss.judge_and_replace('反馈周期',1)
ss.df['流水号'] = list(map(changetype2, ss.df['流水号']))
ss.df['反馈周期_bool'] = ss.df['反馈周期_bool'].astype(int)      #bool类型装换为数值1/0
pp = myModule.sort_by_index(ss.get_total_ratio([['反馈延期率' ,'反馈周期_bool' , '流水号','/']]))
myModule.myPlot(pp).plot('line')

/home/jiankong/jupyter/chart/bin/supplment_analysis/myModule.py:246: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/jiankong/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/jiankong/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stab

### 反馈延期项目列表

In [54]:
ss = myModule.format(df1,'week_number',['流水号','个性化分析名称','信息负责人','下单时间','评估反馈时间','反馈周期'])
ss.select_by_value('week_number',stat_week)
ss.judge_and_replace('反馈周期',1)
ss.select_by_value('反馈周期_bool',True)
tt = myModule.format(ss.df,'week_number',['流水号','个性化分析名称','信息负责人','下单时间','评估反馈时间','反馈周期']).df
tt.sort_values(by = '反馈周期',axis = 0,ascending = False)

,week_number,流水号,个性化分析名称,信息负责人,下单时间,评估反馈时间,反馈周期
252,10月2周,20181011-011,BJ181222-02遗传所1个植物全长转录组只分析任务单-02,刘永强,2018-10-11,2018-10-15,4


## 在线项目数统计

In [55]:
ss = myModule.format(df3 ,'信息负责人' ,['流水号'])
pp = ss.groupby('count')
myModule.myPlot(pp).plot('bar')

# 交付情况统计

## 完成项目数统计

### 整体完成项目数趋势图

In [56]:
ss = myModule.format(df4 ,'week_number' ,['实际交付时间'])
pp = myModule.sort_by_index(ss.groupby('count'))
myModule.myPlot(pp).plot('line')

### 个人完成项目数统计

In [57]:
ss = myModule.format(df4 ,'week_number' ,['实际交付时间'])
ss.add_group('信息负责人')
pp = myModule.sort_by_index(ss.groupby('count'))
myModule.myPlot(pp).plot('stack_bar')

## 完成周期统计

### 整体完成周期趋势图

In [58]:
# 实际完成平均周期，建议添加最长完成周期、25%、50%、80%项目完成周期的指标
ss = myModule.format(df4 ,'week_number' ,['实际周期'])
pp = myModule.sort_by_index(ss.groupby('mean'))
myModule.myPlot(pp).plot('line')

### 个人完成周期统计

In [59]:
df4_fx = df4[df4['信息负责人'].isin(fenxi)] 
ss = myModule.format(df4_fx,'信息负责人' ,['实际周期'])
pp = ss.groupby('mean')
myModule.myPlot(pp).plot('bar')

# 延期情况统计

## 延期率趋势图

In [60]:
ss = myModule.format(df5 ,'week_number' ,['流水号','延期天数'])
ss.judge_and_replace('延期天数',1)
ss.df['流水号'] = list(map(changetype2, ss.df['流水号']))
ss.df['延期天数_bool'] = ss.df['延期天数_bool'].astype(int)      #bool类型装换为数值1/0
pp = myModule.sort_by_index(ss.get_total_ratio([['项目延期率' ,'延期天数_bool' , '流水号','/']]))
myModule.myPlot(pp).plot('line')

/home/jiankong/jupyter/chart/bin/supplment_analysis/myModule.py:246: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/jiankong/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/jiankong/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stab

## 延期项目列表

In [61]:
ss = myModule.format(df5,'week_number',['流水号','个性化分析名称','信息负责人','评估反馈时间','预计交付时间','实际交付时间','延期天数'])
ss.select_by_value('week_number',stat_week)
ss.judge_and_replace('延期天数',1)
ss.select_by_value('延期天数_bool',True)
tt = myModule.format(ss.df,'week_number',['流水号','个性化分析名称','信息负责人','评估反馈时间','预计交付时间','实际交付时间']).df
tt.sort_values(by = '预计交付时间',axis = 0,ascending = False)

,week_number,流水号,个性化分析名称,信息负责人,评估反馈时间,预计交付时间,实际交付时间
178,10月2周,20180927-006,PM-ZJ180327-01浙医二院35个人普通转录组建库测序过滤任务单-快速产品-01,屈昂,2018-09-30,2018-10-14,0
217,10月2周,20181009-009,PM-YWJS180225-01徐州市中心医院20个大鼠粪便宏基因组建库测序过滤任务单-快速产品,冷雪,2018-10-09,2018-10-12,0
230,10月2周,20181010-003,BJ180998-02中药所1个药用动物S4基因组denovo只分析任务单-02,杨伟飞,2018-10-08,2018-10-12,0
105,10月2周,20180917-002,SN170011-04西北农林2个拟南芥样本4个HiC文库测序分析任务单-07,范旭宁,2018-09-17,2018-10-09,0
121,10月2周,20180919-008,ANHL160027东北师大水稻Hi-C-03,范旭宁,2018-09-20,2018-10-08,0


# 7日完成率统计

##  7日完成率趋势图

In [62]:
ss = myModule.format(df4 ,'week_number' ,['流水号','实际周期'])
ss.judge_and_replace('实际周期',7)   # 建议添加小于号
ss.df['流水号'] = list(map(changetype2, ss.df['流水号']))
ss.df['实际周期_bool'] = ss.df['实际周期_bool'].astype(int)      #bool类型装换为数值1/0
pp = myModule.sort_by_index(ss.get_total_ratio([['7日完成率' ,'实际周期_bool' , '流水号','/']]))
myModule.myPlot(pp).plot('line')

/home/jiankong/jupyter/chart/bin/supplment_analysis/myModule.py:246: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/jiankong/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/jiankong/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stab

## 实际周期>7日项目列表

In [63]:
ss = myModule.format(df4,'week_number',['流水号','个性化分析名称','信息负责人','实际周期'])
ss.select_by_value('week_number',stat_week)
ss.judge_and_replace('实际周期',7)
ss.select_by_value('实际周期_bool',True)
tt = myModule.format(ss.df,'week_number',['流水号','个性化分析名称','信息负责人','实际周期']).df
tt.sort_values(by = '实际周期',axis = 0,ascending = False)   #预计周期从大到小排序

,week_number,流水号,个性化分析名称,信息负责人,实际周期
124,10月2周,20180920-003,PM-BJ170057-01、02/03北京蛋白质组研究中心4个小鼠10XGenomics文...,姚盟成,22
136,10月2周,20180921-007,BJ181222-02遗传所1个植物全长转录组只分析任务单-01,刘永强,16
179,10月2周,20180927-007,BJ180998-02中药所1个药用动物S4基因组denovo只分析任务单,马潇,14
172,10月2周,20180926-011,SD170144-04生物所24个爪蟾有参转录组建库测序分析任务单-05,陈碧雯,13
174,10月2周,20180927-002,.南通大学300个大鼠神经元单细胞lncRNA建库测序分析任务单-VIP-04,赵倩,13
169,10月2周,20180926-008,JL180047-01中国农业科学院特产研究所18个水貂单细胞无参转录组只分析任务单-02,何胜,12
175,10月2周,20180927-003,BJ170165-02中科院动物所10个小鼠单细胞甲基化建库任务单-04,韩少怀,12
193,10月2周,20180930-001,SD170077-01山东大学齐鲁医院王刚的6个人单基因病外显子建库测序分析任务单-01,解飞,12
159,10月2周,20180925-010,PM-YWHB180242-02华科协和医院1个人外显子建库测序分析任务单-快速产品-01,昶吉阳,11
185,10月2周,20180928-006,PM-BJ180911-01中国农业科学院北京畜牧所12个牛单细胞全转录组建库测序分析任务单...,冯敬岩,11
